In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plot_models_v_mouse as bp
import model_policies as models
from sklearn.model_selection import train_test_split
import conditional_probs as cprobs
import resample_and_model_reps as reps
import model_fitting as fit
from theory import calculate_escape

ModuleNotFoundError: No module named 'ssm'

In [ ]:
data = pd.read_csv(os.path.join('mouse_data.csv'))
data.head()
probs='80-20' # P(high)-P(low)
seq_nback=3 # history length for conditional probabilites
train_prop=0.7 # for splitting sessions into train and test
seed = np.random.randint(1000) # set seed for reproducibility

data = data.loc[data.Condition==probs] # segment out task condition
data = cprobs.add_history_cols(data, seq_nback) # set history labels up front
data.head()
# train_session_ids, test_session_ids = train_test_split(data.Session.unique(), 
#                                                        train_size=train_prop, random_state=seed) # split full df for train/test

In [ ]:
session_ids = data.Session.unique()
# session_ids = ['m1_63','m1_64','m1_65','m1_67', 'm1_92', 'm1_16', 'm1_17', 'm1_18']
print(session_ids)

In [ ]:

data['block_pos_rev'] = data['blockTrial'] - data['blockLength'] # reverse block position from transition
data['model']='mouse'
data['highPort'] = data['Decision']==data['Target'] # boolean, chose higher probability port

train_features, _, _ = reps.pull_sample_dataset(session_ids, data)
test_features, _, block_pos_core = reps.pull_sample_dataset(session_ids, data)
print(block_pos_core)
block_pos_core.to_csv('./block_pos_core.csv', index=False)
bpos_mouse = bp.get_block_position_summaries(block_pos_core)
bpos_mouse['condition'] = 'mouse'
print(bpos_mouse)

In [ ]:
L1 = 1 # choice history
L2 =  5 # choice * reward history
L3 = 0
memories = [L1, L3, L2, 1]

lr = models.fit_logreg_policy(train_features, memories) # refit model with reduced histories, training set
model_probs = models.compute_logreg_probs(test_features, lr_args=[lr, memories])

In [ ]:
print(lr)

In [ ]:
model_choices, model_switches = models.model_to_policy(model_probs, test_features, policy='stochastic')

linear = reps.reconstruct_block_pos(block_pos_core, model_choices, model_switches)
linear_model = bp.get_block_position_summaries(linear)
linear_model['condition'] = 'model' # label model predictions as such
linear_model_v_mouse = pd.concat((bpos_mouse, linear_model)) # agg df with model predictions and mouse data
color_dict = {'mouse': 'gray', 'model': sns.color_palette()[0]}#plot_config['model_seq_col']}
bp.plot_by_block_position(bpos_mouse, subset='condition', color_dict = color_dict)

In [ ]:
for i in range(1):
    print(len(test_features[0][0]))
    # print(model_choices[0])
    # print(model_switches[0])
    # s = 
    # print(model_choices[i])

In [ ]:
history_length = 5
from Lyapunov_Worm_deconstruction import Lyapunov_Worm_Deconstruction
from tqdm import tqdm
dim = 2
varargin = {
    'step_num': 400,  # how many steps to run the brain circuit before executing the next movement
    'tau': np.ones(dim),  # decay time constant
    'weights_in': np.ones(dim) * 1.,  # input weights
    'rs': np.ones(dim) * .5,  #
    'w': np.ones(dim) * 4,  # weight of mutual inhibition
    'k': 7. * np.ones(dim),  # sigmoid center
    'n': 1.0 * np.ones(dim),  # sigmoid slope
    'bi': np.ones(dim) * 5.,  # baseline production
    'dt': 1.2,  # size of timesteps
    'nsf': 0.85,  # noise level
    'w_avg_comp': 0.5,
    'w_std_comp': 0.5
}
choice_list = []
switches_list = []
ALL = 0
num = 0
for session in range(len(test_features)):
    idx = 0
    net = Lyapunov_Worm_Deconstruction(varargin=varargin, dim=dim)
    choice_history_session = test_features[session][0]
    reward_history_session = test_features[session][1]
    choice_list_session = []
    switches_list_session = []
    for trial in tqdm(range(choice_history_session.shape[0])):
        idx += 1
        # print(idx)
        reward_history_list = [[0.],[0.]]
        lf = max(0, trial - history_length)
        choice_history = choice_history_session[lf:trial]
        reward_history = reward_history_session[lf: trial]
        # print((lf, trial+1))
        # reward_history[reward_history==0] = -1
        decay_list = np.arange(0, len(reward_history), 1)
        decay_list = decay_list[::-1]
        decay_list = 2.1*np.exp(-decay_list/1.3)
        # print(decay_list)
        reward_history = reward_history * decay_list 
        reward_history_list[0] += reward_history[choice_history==0].tolist()
        reward_history_list[1] += reward_history[choice_history==1].tolist()
        # print(reward_history_list)
        reward_history = np.array(reward_history_list)
        last_choice = choice_history[-1] if len(choice_history) > 0 else None
        init_point = None
        # print(reward_history)
        if last_choice == 0:
            init_point = [10.,2.]
        elif last_choice == 1:
            init_point = [2., 10.]
        choice= net.decide(history=reward_history, init=True, init_point=init_point)
        choice_list_session.append(choice)
        if trial == 0:
            switches_list_session.append(0)
        else:
            if choice != last_choice:
                switches_list_session.append(1)
            else:
                switches_list_session.append(0)
        I1 = np.mean(reward_history_list[0]) * varargin['w_avg_comp']
        I2 = np.mean(reward_history_list[1]) * varargin['w_avg_comp']
        # print((noise1, noise2))
        noise1 = np.sqrt(varargin['nsf'] **2 + (np.std(reward_history_list[0])*varargin['w_std_comp']) **2) 
        noise2 = np.sqrt(varargin['nsf'] **2 + (np.std(reward_history_list[1])* varargin['w_std_comp']) **2)
        P1, P2 = calculate_escape(I=[I1, I2], noise=[noise1, noise2])
        # print((P1,P2))
        target = choice_history_session[trial]
        LL = (1-target) * np.log(float(P1)) + target * np.log(float(P2))
        num += 1
        # print(LL)
        ALL += LL
    choice_list.append(choice_list_session)
    # print(choice_list)
    switches_list.append(switches_list_session)
        # print(choice)

In [ ]:
print(ALL/num)

In [ ]:
print(len(choice_list[0]))
print(len(switches_list[0]))

In [ ]:
block_pos_model = reps.reconstruct_block_pos(block_pos_core, choice_list, switches_list)
block_pos_model.to_csv('./block_pos_model.csv', index=False)
bpos_model = bp.get_block_position_summaries(block_pos_model)
print(bpos_model)

In [ ]:

bpos_model['condition'] = 'model' # label model predictions as such
bpos_model_v_mouse = pd.concat((bpos_mouse, bpos_model)) # agg df with model predictions and mouse data
color_dict = {'mouse': 'gray', 'model': sns.color_palette()[0]}#plot_config['model_seq_col']}
bp.plot_by_block_position(bpos_model_v_mouse, subset='condition', color_dict = color_dict)

In [ ]:
df_mouse_symm_reference = cprobs.calc_conditional_probs(data, symm=True, 
                                                        action=['Switch']).sort_values('pswitch')

In [ ]:
df_mouse_symm = cprobs.calc_conditional_probs(block_pos_core, symm=True, action=['Switch', 'Decision'])
df_mouse_symm = cprobs.sort_cprobs(df_mouse_symm, df_mouse_symm_reference.history.values)
print(df_mouse_symm)
bp.plot_sequences(df_mouse_symm, alpha=0.5) 

In [ ]:
print()

In [ ]:
symm_cprobs_model = cprobs.calc_conditional_probs(block_pos_model, symm=True, action=['Switch'])
symm_cprobs_model = cprobs.sort_cprobs(symm_cprobs_model, df_mouse_symm.history.values)
print(symm_cprobs_model)
bp.plot_sequences(df_mouse_symm, overlay=symm_cprobs_model, main_label='mouse', overlay_label='model')
bp.plot_scatter(df_mouse_symm, symm_cprobs_model)

In [ ]:
sum = 0
corr = 0
for idx in range(len(choice_list)):
    targets = test_features[idx][0]
    preds = choice_list[idx]
    corr += np.sum(preds==targets) 
    sum += len(preds)
print(corr/sum)

In [ ]:
sum = 0
corr = 0
for idx in range(len(choice_list)):
    targets = model_switches[idx]
    preds = np.array(switches_list[idx])
    corr += np.sum(preds==targets) 
    sum += len(preds)
print(switches_list[0])
print(model_switches[0])
print(corr/sum)

In [ ]:
print(np.sum(block_pos_model['Switch'] == data['Switch']) / len(data))
print(np.sum(block_pos_model['Decision'] == data['Decision']) / len(data))                                                                                  

In [ ]:
a = np.sum((block_pos_model['Switch'] == 0 ) & (data['Switch'] == 0)) / np.sum(data['Switch'] == 0)
b = np.sum((block_pos_model['Switch'] == 1 ) & (data['Switch'] == 0)) / np.sum(data['Switch'] == 0)
c = np.sum((block_pos_model['Switch'] == 0 ) & (data['Switch'] == 1)) / np.sum(data['Switch'] == 1)
d = np.sum((block_pos_model['Switch'] == 1 ) & (data['Switch'] == 1)) / np.sum(data['Switch'] == 1)
print(a.round(3)) 
print(b.round(3))
print(c.round(3))
print(d.round(3))

In [ ]:
a = np.sum((block_pos_model['Decision'] == 0 ) & (data['Decision'] == 0)) / np.sum(data['Decision'] == 0)
b = np.sum((block_pos_model['Decision'] == 1 ) & (data['Decision'] == 0)) / np.sum(data['Decision'] == 0)
c = np.sum((block_pos_model['Decision'] == 0 ) & (data['Decision'] == 1)) / np.sum(data['Decision'] == 1)
d = np.sum((block_pos_model['Decision'] == 1 ) & (data['Decision'] == 1)) / np.sum(data['Decision'] == 1)
print(a.round(3))
print(b.round(3))
print(c.round(3))
print(d.round(3))
